In [1]:
from pathlib import Path
file = Path("../../final_dataset") / "train.jsonb"

In [ ]:
import sys
sys.path.append("../")

In [2]:
from article_utils import create_df
df = create_df(file)
df.Name = "final"

ModuleNotFoundError: No module named 'article_utils'

In [ ]:
from article_utils import create_exploratory_plots
create_exploratory_plots(df,save=True)